In [141]:
import sys
sys.path.append("/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/_scripts")

from case_edits.recipes.weather import input
from case_edits.ezcase import EzCase

from epw import epw

import seaborn as sns

import pandas as pd
import numpy as np

from weather.joint_analysis import JointAnalysis, N_DATETIME_COLS
from weather.irrelevant_quantities import irrelevant_qois
import plotly.express as px

import plotly.graph_objects as go
import datetime

from helpers.plot_colors import get_plotly_colors

In [156]:
ja = JointAnalysis()

ja.update_qois(["Dry Bulb Temperature", "Wind Speed" ])#"Wind Direction"
ja.create_histogram_df()
ja.freqs_df.head()

df = ja.summer_df
df["Date"] = pd.to_datetime(
            ja.summer_df[["Year", "Month", "Day"]]
        )

dgroup_values = np.linspace(0, 1, ja.nfreqs+1)
df["Distribution Groups"] = np.full(len(df), dgroup_values[0])

In [109]:
# # filter to find when db between desired.. 
# mask = (df["Dry Bulb Temperature"] > 15.47	) & (df["Dry Bulb Temperature"] < 18.93) & (df["Wind Speed"] > 1.97	) & (df["Wind Speed"] < 3.93)
# # df_joint = df.loc[mask]
# # df_joint = df_joint.reset_index(drop=True)
# mask.value_counts()

In [157]:
mask = True

# ignore "Occurences count"
freqs_df = ja.freqs_df.iloc[:, :-1]

for ix in list(range(ja.nfreqs)):
    for key, value in freqs_df.loc[(ix, "min")].iteritems(): 
        mask &= df[key] > value
    for key, value in freqs_df.loc[(ix, "max")].iteritems(): 
        mask &= df[key] < value
    
    if ix == 0:
        df.loc[mask, "Distribution Groups"] = dgroup_values[1]
    else:
        df.loc[mask, "Distribution Groups"] = dgroup_values[0]
    mask = True

In [158]:
df["Distribution Groups"].unique()

array([0. , 0.1])

In [159]:
hours = list(df["Hour"].unique())
dates = list(df["Date"].unique())
arr = np.zeros((len(hours), len(dates)))

for ix, hour in enumerate(hours):
    mask = df["Hour"] == hour
    df1 = df.loc[mask]
    res = df1.reset_index(drop=True)["Distribution Groups"]
    arr[ix, :] = res


In [160]:
colors, _ = get_plotly_colors(n_colors=len(dgroup_values))
colors
colorscale = [[val, col] for val, col in zip(dgroup_values, colors)]
colorscale

[[0.0, 'rgb(48, 18, 59)'],
 [0.1, 'rgb(67, 88, 197)'],
 [0.2, 'rgb(60, 153, 249)'],
 [0.30000000000000004, 'rgb(29, 213, 203)'],
 [0.4, 'rgb(73, 246, 131)'],
 [0.5, 'rgb(164, 252, 59)'],
 [0.6000000000000001, 'rgb(223, 218, 54)'],
 [0.7000000000000001, 'rgb(252, 164, 48)'],
 [0.8, 'rgb(238, 90, 18)'],
 [0.9, 'rgb(193, 37, 3)'],
 [1.0, 'rgb(122, 4, 2)']]

In [182]:
xv, yv = np.meshgrid(dates, hours)
xv

array([['2023-07-01T00:00:00.000000000', '2023-07-02T00:00:00.000000000',
        '2023-07-03T00:00:00.000000000', ...,
        '2023-08-29T00:00:00.000000000', '2023-08-30T00:00:00.000000000',
        '2023-08-31T00:00:00.000000000'],
       ['2023-07-01T00:00:00.000000000', '2023-07-02T00:00:00.000000000',
        '2023-07-03T00:00:00.000000000', ...,
        '2023-08-29T00:00:00.000000000', '2023-08-30T00:00:00.000000000',
        '2023-08-31T00:00:00.000000000'],
       ['2023-07-01T00:00:00.000000000', '2023-07-02T00:00:00.000000000',
        '2023-07-03T00:00:00.000000000', ...,
        '2023-08-29T00:00:00.000000000', '2023-08-30T00:00:00.000000000',
        '2023-08-31T00:00:00.000000000'],
       ...,
       ['2023-07-01T00:00:00.000000000', '2023-07-02T00:00:00.000000000',
        '2023-07-03T00:00:00.000000000', ...,
        '2023-08-29T00:00:00.000000000', '2023-08-30T00:00:00.000000000',
        '2023-08-31T00:00:00.000000000'],
       ['2023-07-01T00:00:00.000000000', '20

In [166]:
["a"]

['remaining', 'dist1']

In [174]:
tickvals = np.linspace(start=0, stop=0.1, num=len(dgroup_values))
tickvals

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ])

In [ ]:
ticktext = 

In [175]:
days = pd.Series(df["Date"].unique()).values.astype('datetime64[s]').tolist()

fig = go.Figure(data=go.Heatmap(
        z=arr,
        x=days,
        y=hours,
        xgap=2,
        ygap=2,
        showscale=True,
        colorscale=colorscale))

# fig.update_xaxes(ticklabelmode="period", dtick="M1", tickformat="%b")
ticktext=["remaining", "dist1"]
fig.update_traces(colorbar_orientation='h',
                  colorbar_tickmode='array',
                  colorbar_ticktext=tickvals,
                  colorbar_tickvals=tickvals,
                  colorbar_tickangle=0,
                  selector=dict(type='heatmap')
                 )

fig.show()

In [140]:
ja.freqs_df

Dry Bulb Temperature  Wind Speed  Occurences
index bins                                              
0     min              15.466667    1.966667       263.0
      max              18.933333    3.933333         NaN
1     min              15.466667    0.000000       154.0
      max              18.933333    1.966667         NaN
2     min              12.000000    0.000000       138.0
      max              15.466667    1.966667         NaN
3     min              18.933333    1.966667       135.0
      max              22.400000    3.933333         NaN
4     min              18.933333    5.900000       112.0
      max              22.400000    7.866667         NaN
5     min              18.933333    3.933333       102.0
      max              22.400000    5.900000         NaN
6     min              15.466667    3.933333        95.0
      max              18.933333    5.900000         NaN
7     min              22.400000    3.933333        92.0
      max              25.866667    5.900000         NaN
8     min              12.000000    1.966667        90.0
      max              15.466667    3.933333         NaN
9     min              22.400000    5.900000        58.0
      max              25.866667    7.866667         NaN

In [114]:
# find all hours and days in df_joint, and make them red here => give them a strange z value